In [ ]:
import os 
import sys
main_path = os.path.abspath(os.path.dirname(os.getcwd()))
sys.path.append(main_path)
from models.classic_models import *

# Climatological CSGD calculation

In [17]:
import os 
list_station_names = set(name[:-len(name.split('_')[-1]) -1] for name in os.listdir('../data/csvs_for_classic_methods'))
len(list_station_names)

130

In [ ]:
import os 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
csvs_dir_ensemble = os.path.abspath(os.path.join('..','data','csvs_for_classic_methods'))

params_0_list = [(l,m,n,p) for l in [0.005,0.01,0.015,0.02] for m in [0.01,0.02,0.05,0.1] for n in np.arange(1,2,0.25) for p in [0.01,0.05,0.1,0.15]]

total_n = 0
total_crps = 0
total_rmse = 0
total_bs = 0
for station_name in list_station_names:
    ensemb_train_file = os.path.join(csvs_dir_ensemble ,station_name + '_train.csv')
    ensemb_test_file = os.path.join(csvs_dir_ensemble ,station_name + '_test.csv')
    ensemb_train_df = pd.read_csv(ensemb_train_file).dropna()
    ensemb_test_df = pd.read_csv(ensemb_test_file).dropna()
    ground_truth_train = ensemb_train_df.iloc[:,1].values
    ground_truth_test = ensemb_test_df.iloc[:,1].values
    if len(ground_truth_test) == 0:
        print(f'No rain in {station_name}')
        continue
    climat_csgd = CSGD(verbose=False)
    params0 = climat_csgd.calc_initial_values(ground_truth = ground_truth_train, method='paper')
    climat_csgd.fit_climatological(ground_truth = ground_truth_train, init_params = params0)#,params_0 = [0.01,0.02,1.5,0.1])
    crps, rmse, mse, bs = climat_csgd.predict(ground_truth_test, metric = 'all')
    total_n += len(ground_truth_test)
    total_crps += crps*len(ground_truth_test)
    total_rmse += rmse*len(ground_truth_test)
    total_bs += bs*len(ground_truth_test)
    print(f'{station_name} RMSE: {rmse}')

crps_value = total_crps/total_n
rmse_value = total_rmse/total_n
bs_value = total_bs/total_n
print(total_n)
print(f'Climatological. CRPS:{crps_value} RMSE:{rmse_value} BS:{bs_value}')

ABONACOP RMSE: 0.004818741463080374
AGUAMANSA_MA RMSE: 0.04711091785883541
ALCALA RMSE: 1.106536583259492e-05
ANAVISTH RMSE: 0.005604546646910361
ARAYA RMSE: 0.0025364214184887377
ARICO_01 RMSE: 9.11674621724473e-06
BADAJTH RMSE: 0.0034420289855072467
BAILADERO RMSE: 0.06334395850248965
BENIJTH RMSE: 0.023664077605660187
BVISTA02 RMSE: 0.016235242476570693
C314Z_VALLEHERMOSO-ALTOIGUALERO RMSE: 0.033878813925978524
C316I_ARURECEMENTERIO RMSE: 0.028706078242470824
C317B_AGULO-JUEGOBOLAS RMSE: 0.07203861106832861
C319W_VALLEHERMOSO-DAMA RMSE: 0.00019797045823442954
C328W_HERMIGUA-DEPÓSITOAYUNTAMIENTO RMSE: 0.007267324763979238
C329B_LAGOMERA_AEROPUERTO RMSE: 0.00024328563505693405
C329Z_SANSEBASTIÁNDELAGOMERA RMSE: 0.0004782794794498605
C406G_CAÑADASPARADOR RMSE: 0.0007392407502187836
C412N_CITFAGRO_89_CHAVAO RMSE: 0.0018303274062725534
C415A_CITFAGRO_64_HOYOS RMSE: 0.013861543282643156
C417J_CITFAGRO_74_CHIO RMSE: 0.0037138098601114276
C418I_CITFAGRO_05_GUIAISO1 RMSE: 0.00315227287011657

Calculated hourly:


In [ ]:
import os 
import pandas as pd
import numpy as np
import warnings
from collections import defaultdict
warnings.filterwarnings('ignore')

csvs_dir_ensemble = os.path.abspath(os.path.join('..','data','csvs_for_classic_methods'))
params_0_list = [(l,m,n,p) for l in [0.005,0.01,0.015,0.02] for m in [0.01,0.02,0.05,0.1] for n in np.arange(1,2,0.25) for p in [0.01,0.05,0.1,0.15]]

total_n = defaultdict(int)
total_crps = defaultdict(float)
total_rmse = defaultdict(float)
total_bs = defaultdict(float)
for station_name in list_station_names:
    ensemb_train_file = os.path.join(csvs_dir_ensemble ,station_name + '_train.csv')
    ensemb_test_file = os.path.join(csvs_dir_ensemble ,station_name + '_test.csv')
    ensemb_train_df = pd.read_csv(ensemb_train_file,parse_dates=['date']).dropna()
    ensemb_test_df = pd.read_csv(ensemb_test_file,parse_dates=['date']).dropna()

    ground_truth_train = ensemb_train_df.iloc[:,1].values
    climat_csgd = CSGD(verbose=False)
    params0 = climat_csgd.calc_initial_values(ground_truth = ground_truth_train, method='paper')
    climat_csgd.fit_climatological(ground_truth = ground_truth_train, init_params = params0)#,params_0 = [0.01,0.02,1.5,0.1])
    for i in range(6,30):
        hour = i if i < 24 else (i - 24)
        gt_test_hour = ensemb_test_df.loc[ensemb_test_df['date'].dt.hour == hour,ensemb_test_df.columns[1]].values
        crps, rmse, mse, bs = climat_csgd.predict(gt_test_hour, metric = 'all')
        
        total_n[i] += len(gt_test_hour)
        total_crps[i] += crps*len(gt_test_hour)
        total_rmse[i] += rmse*len(gt_test_hour)
        total_bs[i] += bs*len(gt_test_hour)

crps_climat_value = {i:total_crps[i]/total_n[i] for i in total_crps.keys()}
rmse_climat_value = {i:total_rmse[i]/total_n[i] for i in total_crps.keys()}
bs_climat_value = {i:total_bs[i]/total_n[i] for i in total_crps.keys()}
print(sum(total_n.values()))
print('Mean overall RMSE (comprobation):',sum(total_rmse.values())/sum(total_n.values()))
print(f'Climatological. \n CRPS:{crps_climat_value}\n RMSE:{rmse_climat_value}\n BrierScore:{bs_climat_value}')

139922
Mean overall RMSE (comprobation): 0.019751875750723295
Climatological. 
 CRPS:{6: 0.017388973759246747, 7: 0.02071459585170727, 8: 0.019163449785138044, 9: 0.01608812539168957, 10: 0.020356949722375256, 11: 0.01842325807284551, 12: 0.026944481835117304, 13: 0.03391330324368989, 14: 0.02463188308222405, 15: 0.014297282637880054, 16: 0.014886215109004118, 17: 0.011734821174125127, 18: 0.01698395743068431, 19: 0.02110732835642871, 20: 0.020951706615750788, 21: 0.01489150362683552, 22: 0.012946843380200924, 23: 0.015773758417082534, 24: 0.024581913783453382, 25: 0.025518036510403633, 26: 0.017882215114604496, 27: 0.014730802418092265, 28: 0.015823540712450943, 29: 0.013465947111223703}
 RMSE:{6: 0.018299157123635966, 7: 0.021633263623214244, 8: 0.020057045783834482, 9: 0.016954790892599082, 10: 0.021284194295431488, 11: 0.01926475092355835, 12: 0.02786607551312786, 13: 0.03483935770368473, 14: 0.025504747286307878, 15: 0.015055485028473512, 16: 0.015631640469456344, 17: 0.0124902388

# Predictive CSGD calculation

Using SLSQP with constraints:

In [ ]:
import os 
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

csvs_dir_ensemble = os.path.abspath(os.path.join('..','data','csvs_for_classic_methods'))
params_0_list = [(l,m,n,p) for l in [0.005,0.01,0.015,0.02] for m in [0.01,0.02,0.05,0.1] for n in np.arange(1,2,0.25) for p in [0.01,0.05,0.1,0.15]]

total_crps = 0
total_rmse = 0
total_bs = 0
total_n = 0
estaciones_pesos = {}
for i,station_name in enumerate(list_station_names):
    print(f'·{i + 1}/{len(list_station_names)} {station_name}:')

    ensemb_train_file = os.path.join(csvs_dir_ensemble ,station_name + '_train.csv')
    ensemb_val_file = os.path.join(csvs_dir_ensemble ,station_name + '_val.csv')
    ensemb_test_file = os.path.join(csvs_dir_ensemble ,station_name + '_test.csv')

    ensemb_train_df = pd.read_csv(ensemb_train_file).dropna()
    ensemb_val_df = pd.read_csv(ensemb_val_file).dropna()
    ensemb_test_df = pd.read_csv(ensemb_test_file).dropna()

    ground_truth_train = ensemb_train_df.iloc[:,1].values
    wrf_ensemble_train = ensemb_train_df.iloc[:,2:].values
    ground_truth_val = ensemb_val_df.iloc[:,1].values
    wrf_ensemble_val = ensemb_val_df.iloc[:,2:].values
    ground_truth_test = ensemb_test_df.iloc[:,1].values
    wrf_ensemble_test = ensemb_test_df.iloc[:,2:].values
    n_station = len(ground_truth_test)
    best_crps_val = 1
    for params_0 in params_0_list:
        pred_csgd = PredictiveCSGD(verbose=False)
        pred_csgd.fit(ground_truth_train,wrf_ensemble_train, params_0 = params_0)#,params_0 = [0.01,0.02,1.5,0.1])
        crps_val = pred_csgd.predict(ground_truth_val,wrf_ensemble_val)
        if crps_val < best_crps_val:
            best_crps_val = crps_val
            best_pred_csgd = pred_csgd
            best_init_params = params_0

    crps_test, rmse_test, mse_test, bs_test = best_pred_csgd.predict(ground_truth_test,wrf_ensemble_test, metrics = 'all')
    print(f' Best params_init: {best_init_params}, best_params_fitted: {best_pred_csgd.fitted_params} \n crps: {crps_test} rmse: {rmse_test} bs: {bs_test}')
    total_n += n_station
    total_crps += crps_test * n_station
    total_rmse += rmse_test * n_station
    total_bs += bs_test * n_station
    estaciones_pesos[station_name] = {'pred':best_pred_csgd.fitted_params,'climat':best_pred_csgd.fitted_climat_params}

crps_final = total_crps/total_n
rmse_final = total_rmse / total_n
bs_final = total_bs / total_n
print(f' crps_final = {crps_final} rmse = {rmse_final} bs = {bs_final}')

import pickle
# Guardamos el diccionario en un archivo .pkl usando pickle
with open('PredCSGD_pesos.pkl', 'wb') as f:
    pickle.dump(estaciones_pesos, f)

·1/130 ABONACOP:
 Best params_init: (0.02, 0.05, 1.5, 0.15), best_params_fitted: [0.27324648 0.06370034 1.50725016 0.10804025] 
 crps: 0.004532398105421011 rmse: 0.007109313705034377 bs: 0.00096208223634352
·2/130 AGUAMANSA_MA:
 Best params_init: (0.005, 0.1, 1.0, 0.05), best_params_fitted: [0.001      0.2963683  0.41622599 0.67510064] 
 crps: 0.03824864328483176 rmse: 0.04353179257305395 bs: 0.009714594207029893
·3/130 ALCALA:
 Best params_init: (0.005, 0.1, 1.75, 0.15), best_params_fitted: [0.82533301 0.09251216 1.71554048 0.46881239] 
 crps: 6.173941195900703e-06 rmse: 2.2266673601165667e-05 bs: 3.724091900636249e-17
·4/130 ANAVISTH:
 Best params_init: (0.015, 0.02, 1.75, 0.01), best_params_fitted: [0.31050458 0.01989476 1.74536972 0.15632246] 
 crps: 0.0052042790403563875 rmse: 0.006853590623515255 bs: 8.698009842393014e-05
·5/130 ARAYA:
 Best params_init: (0.005, 0.02, 1.0, 0.15), best_params_fitted: [0.17758014 0.0191394  0.97816424 0.32106244] 
 crps: 0.00332599147955427 rmse: 0

Calculated hourly:

In [ ]:
import os 
import pandas as pd
import warnings
from collections import defaultdict
import pickle

warnings.filterwarnings('ignore')
csvs_dir_ensemble = os.path.abspath(os.path.join('..','data','csvs_for_classic_methods'))

with open('PredCSGD_pesos.pkl', 'rb') as f:
    estac_pesos = pickle.load(f)

total_crps = defaultdict(float)
total_mse = defaultdict(float)
total_rmse = defaultdict(float)
total_bs = defaultdict(float)
total_n = defaultdict(int)
for station_name in list_station_names:
    ensemb_train_file = os.path.join(csvs_dir_ensemble ,station_name + '_train.csv')
    ensemb_test_file = os.path.join(csvs_dir_ensemble ,station_name + '_test.csv')

    ensemb_train_df = pd.read_csv(ensemb_train_file).dropna()
    ensemb_test_df = pd.read_csv(ensemb_test_file, parse_dates=['date']).dropna()

    ground_truth_train = ensemb_train_df.iloc[:,1].values
    wrf_ensemble_train = ensemb_train_df.iloc[:,2:].values

    n_station = len(ensemb_test_df)
    best_crps_val = 1
    
    pred_csgd = PredictiveCSGD(params = estac_pesos[station_name]['pred'], forecast_climat_mean = np.mean(wrf_ensemble_train),
                               climat_params= estac_pesos[station_name]['climat'] ,verbose=False)
    for i in range(6,30):
        hour = i if i < 24 else (i - 24)
        gt_test_hour = ensemb_test_df.loc[ensemb_test_df['date'].dt.hour == hour,ensemb_test_df.columns[1]].values
        wrf_ensemb_test_hour = ensemb_test_df.loc[ensemb_test_df['date'].dt.hour == hour, ensemb_test_df.columns[2:]].values
        crps, rmse, mse, bs = pred_csgd.predict(gt_test_hour, wrf_ensemb_test_hour, metrics = 'all')
        
        total_n[i] += len(gt_test_hour)
        total_crps[i] += crps*len(gt_test_hour)
        total_rmse[i] += rmse*len(gt_test_hour)
        total_mse[i] += mse*len(gt_test_hour)
        total_bs[i] += bs*len(gt_test_hour)

crps_pred_value = {i:total_crps[i]/total_n[i] for i in total_crps.keys()}
mse_pred_value = {i:total_mse[i]/total_n[i] for i in total_crps.keys()}
rmse_pred_value = {i:total_rmse[i]/total_n[i] for i in total_crps.keys()}
bs_pred_value = {i:total_bs[i]/total_n[i] for i in total_crps.keys()}
print(sum(total_n.values()))
print('Mean overall rmse (comprobation):',sum(total_rmse.values())/sum(total_n.values()))
print(f'PredCSGD. \n CRPS:{crps_pred_value}\n RMSE:{rmse_pred_value}\n BrierScore:{bs_pred_value} \n MSE:{mse_pred_value}')

139922
Mean overall rmse (comprobation): 0.02076011882007269
PredCSGD. 
 CRPS:{6: 0.015456664695547077, 7: 0.01862197446258743, 8: 0.01756601645411942, 9: 0.014064996993671754, 10: 0.017219717875128167, 11: 0.016174786094554965, 12: 0.02446416001708243, 13: 0.030751793243724866, 14: 0.02171174776512049, 15: 0.01277544429822616, 16: 0.013903181194665907, 17: 0.011144957757063114, 18: 0.0162341447346739, 19: 0.019350181100656373, 20: 0.01871616718428225, 21: 0.01326042942439428, 22: 0.011697841132110011, 23: 0.014530600274004912, 24: 0.023203632588669705, 25: 0.023771876684760364, 26: 0.016016758925065697, 27: 0.013463647048575625, 28: 0.014790831133560969, 29: 0.012524187195792819}
 RMSE:{6: 0.01942013713969002, 7: 0.021941141925927974, 8: 0.021118699580646636, 9: 0.01742790666161281, 10: 0.020570777386847507, 11: 0.019921375588301843, 12: 0.02870909286061271, 13: 0.03492671723978654, 14: 0.025892112299543857, 15: 0.01632994078417052, 16: 0.017526746470673402, 17: 0.014917249607818461, 

## Analog Ensemble

In [ ]:
from classic_models import *
import os 
import pandas as pd
import warnings
from properscoring import crps_ensemble
warnings.filterwarnings('ignore')
csvs_dir_ensemble = os.path.abspath(os.path.join('..','data','csvs_for_classic_methods'))

total_n = 0
total_crps = 0
total_crps_ensemb = 0
total_rmse = 0
total_mse = 0
total_bs = 0
for idx,station_name in enumerate(list_station_names):
    print(f'·{idx}/{len(list_station_names)}.{station_name}:')
    ensemb_train_file = os.path.join(csvs_dir_ensemble ,station_name + '_train.csv')
    ensemb_val_file = os.path.join(csvs_dir_ensemble ,station_name + '_val.csv')
    ensemb_test_file = os.path.join(csvs_dir_ensemble ,station_name + '_test.csv')
    
    ensemb_train_df = pd.read_csv(ensemb_train_file).dropna()
    ensemb_val_df = pd.read_csv(ensemb_val_file).dropna()
    ensemb_test_df = pd.read_csv(ensemb_test_file).dropna()
    ground_truth_train = ensemb_train_df.iloc[:,1].values
    ground_truth_test = ensemb_test_df.iloc[:,1].values

    AnEn = AnalogEnsemble(t_window=3)
    AnEn.fit(X_train = ensemb_train_df.iloc[:,2:], y_train = ensemb_train_df.iloc[:,1], X_val = ensemb_val_df.iloc[:,2:], y_val = ensemb_val_df.iloc[:,1], n_members_options=[25,35,50]) 
    crps, rmse, mse, bs = AnEn.predict(X_test = ensemb_test_df.iloc[:,2:], y_test = ensemb_test_df.iloc[:,1])
    crps_ensemb = np.mean(crps_ensemble(ground_truth_test, ensemb_test_df.iloc[:,2:].values))
    total_n += len(ground_truth_test)
    total_crps += crps*len(ground_truth_test)
    total_crps_ensemb += crps_ensemb*len(ground_truth_test)
    total_rmse += rmse*len(ground_truth_test)
    total_mse += mse*len(ground_truth_test)
    total_bs += bs*len(ground_truth_test)
    print(f' CRPS: {crps} CRPS_ensemb: {crps_ensemb}')
    print(f' BS {bs} RMSE {rmse} MSE {mse}')
    print('')
crps_value = total_crps/total_n
rmse_value = total_rmse/total_n
mse_value = total_mse/total_n
bs_value = total_bs/total_n
print(total_crps_ensemb/total_n)
print(total_n)
print(f'Climatological. CRPS:{crps_value} RMSE:{rmse_value} MSE:{mse_value} BS:{bs_value}')

·0/130.ABONACOP:
Mejores pesos: [1.02923164 0.69814124 0.99522092 1.05479309 1.00323908 0.9578255
 1.0669228  1.06401223 1.05935098 0.92934417 1.0697859  0.9902395
 0.91324662 1.0407546  0.76070251 1.07913821 1.06118349 1.08864994
 0.99403227 1.08005602 1.08494014 0.96604577 1.01424093 1.02155932
 0.97734314]
Mejor número de miembros: 50
 CRPS: 0.004435036231884101 CRPS_ensemb: 0.005477001769390704
 BS 0.0008478260869565219 RMSE 0.010860507246376829 MSE 0.003937228260869512

·1/130.AGUAMANSA_MA:
Mejores pesos: [0.93368158 1.04171911 1.09470387 0.79200651 0.93234571 1.03543342
 0.84512121 1.27288825 0.65734631 0.85661231 1.09565863 1.15039114
 0.96526429 1.15290847 1.01671875 0.71282961 1.01655623 1.34711341
 1.06569298 1.11994176 0.96279281 1.14400214 0.90107451 1.17028924
 0.71690775]
Mejor número de miembros: 50
 CRPS: 0.038106311926605435 CRPS_ensemb: 0.0369298660250782
 BS 0.009890275229357811 RMSE 0.04935596330275235 MSE 0.04950649908256879

·2/130.ALCALA:
Mejores pesos: [1.005149

Poor results for Analog Ensemble.